### Imports

In [256]:
import pandas as pd
from auto_ts import auto_timeseries
import dill
import talib
import numpy as np
from autots import AutoTS

In [257]:
# Evito que ciertas columnas se transformen a notacion cientifica en las predicciones
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

In [258]:
columns = [
    'Open_time',
    'Open',
    'High',
    'Low',
    'Close',
]

columns_to_model = [
    # 'Open_time',
    'Open',
    'High',
    'Low',
    'SMA_20',
    'EMA_20',
    'Upper_Band',
    'Middle_Band',
    'Lower_Band',
    'RSI',
    'MACD',
    'Signal',
    'ADX',
    'SlowK',
    'SlowD',
    'CCI',
    'ATR',
]

### Model

In [259]:
complete_dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv') 
complete_dataset['Open_time'] = pd.to_datetime(complete_dataset['Open_time'], errors='coerce')
columns = ['Open', 'High', 'Low', 'Close']

In [260]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
features = complete_dataset[columns]
# features_scaled = scaler.fit_transform(features)

# feature_dataset_scaled = pd.DataFrame(features_scaled, columns=columns)

display(features)

,Open,High,Low,Close
0,28.84,30.26,27.50,27.71
1,27.72,28.38,26.14,26.31
2,26.31,28.59,26.11,27.28
3,27.28,28.99,27.13,28.62
4,28.61,32.33,28.50,31.94
...,...,...,...,...
951,7.12,7.40,7.01,7.30
952,7.30,7.47,7.06,7.12
953,7.12,7.29,6.98,6.99
954,6.99,7.19,6.89,6.98


In [261]:
features['timestamp'] = pd.to_datetime(complete_dataset['Open_time'], errors='coerce')
features = features[['timestamp'] + columns]

features = features.set_index('timestamp')
display(features)

,Open,High,Low,Close
timestamp,,,,
2021-09-27,28.84,30.26,27.50,27.71
2021-09-28,27.72,28.38,26.14,26.31
2021-09-29,26.31,28.59,26.11,27.28
2021-09-30,27.28,28.99,27.13,28.62
2021-10-01,28.61,32.33,28.50,31.94
...,...,...,...,...
2024-05-05,7.12,7.40,7.01,7.30
2024-05-06,7.30,7.47,7.06,7.12
2024-05-07,7.12,7.29,6.98,6.99


In [262]:
train = features[:-10]
val = features[-10:]

In [263]:
n_days_to_predict = 10

In [264]:
# Crear un diccionario para almacenar los modelos
trained_models = {}

In [265]:
model = AutoTS(
    forecast_length=10,
    frequency='D',
    ensemble='horizontal-max',
    model_list=['ARIMA', 'VARMAX', 'VAR', 'RollingRegression', 'NVAR', 'WindowRegression', 'GluonTS', 'PytorchForecasting', 'GLM'],
    validation_method="backwards", #Similarity
    transformer_list='fast',
    num_validations=3,
    max_generations=15,
    # models_to_validate=0.7, #<= Toma los mejores 0.X% modelos para aplicar el cross validation
    prediction_interval=0.9, #<= Incertidumbre
    metric_weighting={
            'smape_weighting': 1,
            'mae_weighting': 0.5,
            'rmse_weighting': 0.5,
            'made_weighting': 0,
            'mage_weighting': 0,
            'mle_weighting': 0,
            'imle_weighting': 0,
            'spl_weighting': 0,
            'containment_weighting': 0,
            'contour_weighting': 0,
            'runtime_weighting': 0,
            'oda_weighting': 0,
            'wasserstein_weighting': 0,
        }
)


model = model.fit(train)

5 - RollingRegression with avg smape 23.73: 
Model Number: 6 of 79 with model ARIMA for Validation 1
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           17     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -3.40718D+00    |proj g|=  4.33869D+01
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           17     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -3.57133D+00    |proj g|=  8.45746D+01
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           17     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -3.33380D+00    |proj g|=  4.39922D+01
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           17     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -3.34578D+00    |proj 

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.



At iterate    1    f= -3.60517D+00    |proj g|=  5.85959D+00

At iterate    2    f= -3.35215D+00    |proj g|=  8.69815D+00

At iterate    3    f= -3.42324D+00    |proj g|=  2.87931D+00

At iterate    1    f= -3.38949D+00    |proj g|=  2.85714D+00

At iterate    2    f= -3.60544D+00    |proj g|=  2.22444D+00

At iterate    3    f= -3.35396D+00    |proj g|=  3.89976D+00

At iterate    4    f= -3.42341D+00    |proj g|=  1.71371D+00

At iterate    2    f= -3.38962D+00    |proj g|=  1.52074D+00

At iterate    3    f= -3.60550D+00    |proj g|=  8.11979D-01

At iterate    5    f= -3.42355D+00    |proj g|=  1.39635D+00

At iterate    4    f= -3.35459D+00    |proj g|=  3.52199D+00

At iterate    3    f= -3.38974D+00    |proj g|=  1.32063D+00

At iterate    4    f= -3.60552D+00    |proj g|=  6.55443D-01

At iterate    5    f= -3.35693D+00    |proj g|=  1.86890D+00

At iterate    6    f= -3.42381D+00    |proj g|=  1.51670D+00

At iterate    5    f= -3.60557D+00    |proj g|=  4.15508D-01

At iter


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.

 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.

 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.



At iterate   43    f= -3.45598D+00    |proj g|=  5.22364D-01

At iterate   44    f= -3.45599D+00    |proj g|=  4.24260D-01

At iterate   45    f= -3.45600D+00    |proj g|=  1.94315D-01

At iterate   57    f= -3.39341D+00    |proj g|=  1.07794D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   17     57     92      1     0     0   1.078D-01  -3.393D+00
  F =  -3.3934059680381177     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             

At iterate   46    f= -3.45601D+00    |proj g|=  8.83155D-02

At iterate   47    f= -3.45601D+00    |proj g|=  2.74894D-01

At iterate   48    f= -3.45602D+00    |proj g|=  1

...


At iterate   41    f= -3.96406D+00    |proj g|=  2.43236D+00

At iterate   42    f= -3.96411D+00    |proj g|=  1.26562D+00

At iterate   43    f= -3.96416D+00    |proj g|=  1.24794D+00

At iterate   44    f= -3.96417D+00    |proj g|=  2.83526D+00

At iterate   45    f= -3.96424D+00    |proj g|=  7.47343D-01

At iterate   46    f= -3.96428D+00    |proj g|=  7.11903D-01

At iterate   47    f= -3.96430D+00    |proj g|=  1.59716D+00

At iterate   48    f= -3.96431D+00    |proj g|=  3.31525D+00

At iterate   49    f= -3.96435D+00    |proj g|=  3.22485D+00

At iterate   50    f= -3.96452D+00    |proj g|=  4.97123D+00

At iterate   51    f= -3.96465D+00    |proj g|=  3.42302D+00

At iterate   52    f= -3.96468D+00    |proj g|=  3.81904D-01

At iterate   53    f= -3.96473D+00    |proj g|=  1.11147D-01

At iterate   54    f= -3.96489D+00    |proj g|=  1.82595D-01

At iterate   55    f= -3.96490D+00    |proj g|=  1.46352D+00

At iterate   56    f= -3.96502D+00    |proj g|=  5.82972D-01

At iter


 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



At iterate   60    f= -3.96520D+00    |proj g|=  1.22247D-01

At iterate   61    f= -3.96520D+00    |proj g|=  1.27433D-01
  ys=-2.311E-10  -gs= 6.641E-09 BFGS update SKIPPED

At iterate   62    f= -3.96520D+00    |proj g|=  1.27433D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   16     62    131      2     1     0   1.274D-01  -3.965D+00
  F =  -3.9652044330569121     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
78 - ARIMA with avg smape 3.77: 
Model Number: 79 of 79 with model NVAR for Validation 3



   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


79 - NVAR with avg smape 4.46: 
TotalRuntime missing in 17!
Validation Round: 1
TotalRuntime missing in 0!
Validation Round: 2
TotalRuntime missing in 0!
Validation Round: 3
TotalRuntime missing in 0!
Model Number: 1 with model Ensemble in generation 0 of Horizontal Ensembles
Model Number: 2 with model Ensemble in generation 0 of Horizontal Ensembles


In [266]:
prediction = model.predict()
forecast = prediction.forecast
print(forecast)

            Open  High  Low  Close
2024-04-30  6.84  6.99 6.56   6.77
2024-05-01  6.84  6.91 6.70   6.70
2024-05-02  6.89  6.83 6.82   6.63
2024-05-03  6.91  6.76 6.80   6.56
2024-05-04  6.93  6.69 6.95   6.49
2024-05-05  7.00  6.62 6.95   6.42
2024-05-06  6.95  6.55 7.03   6.36
2024-05-07  6.94  6.49 7.06   6.30
2024-05-08  6.92  6.42 7.31   6.24
2024-05-09  6.91  6.36 7.40   6.18


#### Indicadores Tecnicos

In [267]:
def calcular_indicadores_tecnicos(historico_precio):
    close_prices = np.array(historico_precio['Close'], dtype=np.double)
    high_prices = np.array(historico_precio['High'], dtype=np.double)
    low_prices = np.array(historico_precio['Low'], dtype=np.double)
    
    # Media Móvil Simple (SMA)
    sma_20 = talib.SMA(close_prices, timeperiod=20)
    
    # Media Móvil Exponencial (EMA)
    ema_20 = talib.EMA(close_prices, timeperiod=20)
    
    # Bandas de Bollinger
    upper_band, middle_band, lower_band = talib.BBANDS(close_prices, timeperiod=20, nbdevup=2, nbdevdn=2)
    
    # Índice de Fuerza Relativa (RSI)
    rsi = talib.RSI(close_prices, timeperiod=14)
    
    # MACD (Convergencia/Divergencia de Medias Móviles)
    macd, signal, _ = talib.MACD(close_prices, fastperiod=12, slowperiod=26, signalperiod=9)
    
    # ADX (Índice Direccional Promedio)
    adx = talib.ADX(high_prices, low_prices, close_prices, timeperiod=14)
    
    # Estocástico
    slowk, slowd = talib.STOCH(high_prices, low_prices, close_prices, fastk_period=14, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    
    # Índice de Canal de Materias Primas 
    cci = talib.CCI(high_prices, low_prices, close_prices, timeperiod=14)
    
    # ATR (Average True Range)
    atr = talib.ATR(high_prices, low_prices, close_prices, timeperiod=14)
    
    indicadores = pd.DataFrame({
        'SMA_20': sma_20,
        'EMA_20': ema_20,
        'Upper_Band': upper_band,
        'Middle_Band': middle_band,
        'Lower_Band': lower_band,
        'RSI': rsi,
        'MACD': macd,
        'Signal': signal,
        'ADX': adx,
        'SlowK': slowk,
        'SlowD': slowd,
        'CCI': cci,
        'ATR': atr
    }, index=historico_precio.index)
    
    return indicadores

In [268]:
# forecast_results = {}

# for column in columns:
#     future_predictions = trained_models[column].predict()
#     forecast_results[column] = future_predictions

# # Mostrar los resultados del pronóstico
# for column, predictions in forecast_results.items():
#     print(f"Pronóstico para la columna '{column}':")
#     display(predictions)
#     print()

#### Guardo el mejor modelo para cada columna

In [269]:
for column, model in trained_models.items():
    # Guarda el modelo en un archivo usando dill
    with open(f"auto_ts_models/{column}.pkl", "wb") as file:
        dill.dump(model, file)

#### Vuelvo a armar el mejor modelo para cada columna

In [270]:
loaded_models = {}

# Itera sobre los archivos guardados
for column in columns:
    if column != 'Open_time':
        # Cargar el modelo desde el archivo
        with open(f"auto_ts_models/{column}.pkl", "rb") as file:
            loaded_model = dill.load(file)
        
        # Guardar el modelo cargado en el diccionario
        loaded_models[column] = loaded_model

#### Prediccion N dias en el futuro

In [271]:
predictions_df = pd.DataFrame(forecast)

# for column, predictions in forecast_results.items():
#     predictions_df[column] = predictions['yhat'].values

# Añadir la columna de timestamp
predictions_df['Open_time'] = val.index

In [272]:
display(predictions_df[['Open_time', 'Close', 'Open', 'High', 'Low']])

,Open_time,Close,Open,High,Low
2024-04-30,2024-04-30,6.77,6.84,6.99,6.56
2024-05-01,2024-05-01,6.70,6.84,6.91,6.70
2024-05-02,2024-05-02,6.63,6.89,6.83,6.82
2024-05-03,2024-05-03,6.56,6.91,6.76,6.80
2024-05-04,2024-05-04,6.49,6.93,6.69,6.95
2024-05-05,2024-05-05,6.42,7.00,6.62,6.95
2024-05-06,2024-05-06,6.36,6.95,6.55,7.03
2024-05-07,2024-05-07,6.30,6.94,6.49,7.06
2024-05-08,2024-05-08,6.24,6.92,6.42,7.31
2024-05-09,2024-05-09,6.18,6.91,6.36,7.40


In [273]:
final_df = pd.concat([complete_dataset[['Open_time', 'Close'] + columns_to_model], predictions_df[['Open_time','Close', 'Open', 'High', 'Low']]], ignore_index=True)

display(final_df)

,Open_time,Close,Open,High,Low,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR
0,2021-09-27,27.71,28.84,30.26,27.50,31.85,30.83,38.30,31.85,25.39,44.11,0.02,0.96,21.20,24.79,33.78,-86.30,4.00
1,2021-09-28,26.31,27.72,28.38,26.14,31.77,30.40,38.44,31.77,25.11,41.83,-0.33,0.70,21.98,16.10,24.47,-109.46,3.88
2,2021-09-29,27.28,26.31,28.59,26.11,31.65,30.10,38.55,31.65,24.74,43.99,-0.52,0.46,22.61,12.88,17.92,-92.93,3.78
3,2021-09-30,28.62,27.28,28.99,27.13,31.62,29.96,38.56,31.62,24.67,46.92,-0.56,0.26,22.99,16.42,15.13,-60.03,3.64
4,2021-10-01,31.94,28.61,32.33,28.50,31.64,30.15,38.58,31.64,24.69,53.42,-0.31,0.14,21.86,34.90,21.40,34.17,3.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
961,2024-05-05,6.42,7.00,6.62,6.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
962,2024-05-06,6.36,6.95,6.55,7.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
963,2024-05-07,6.30,6.94,6.49,7.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
964,2024-05-08,6.24,6.92,6.42,7.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [274]:
indicadores_tecnicos = calcular_indicadores_tecnicos(final_df)
display(indicadores_tecnicos)

,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
961,6.84,6.87,7.39,6.84,6.29,35.99,-0.24,-0.23,27.01,15.87,30.42,-99.98,0.41
962,6.82,6.82,7.41,6.82,6.23,34.91,-0.26,-0.24,27.03,-3.49,15.63,-89.48,0.42
963,6.80,6.77,7.43,6.80,6.16,33.85,-0.27,-0.24,27.06,-22.03,-3.22,-87.64,0.44
964,6.77,6.72,7.45,6.77,6.09,32.80,-0.28,-0.25,27.08,-28.79,-18.10,-66.64,0.49


In [275]:
validation_last_10 = complete_dataset[columns_to_model].tail(10)
ultimos_registros_last_10 = final_df.tail(10)

validation_last_10 = validation_last_10[columns_to_model].reset_index(drop=True)
ultimos_registros_last_10 = ultimos_registros_last_10[columns_to_model].reset_index(drop=True)

diferencia = validation_last_10 - ultimos_registros_last_10

display(diferencia)
# display(ultimos_registros_last_10)
# display(validation_last_10)

,Open,High,Low,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR
0,-0.26,-0.32,-0.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.42,0.03,-0.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.01,0.57,-0.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.36,0.62,0.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.31,0.59,0.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.12,0.78,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.35,0.92,0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.18,0.81,-0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.07,0.76,-0.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.07,0.73,-0.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [276]:
ultimos_indicadores = indicadores_tecnicos.tail(10)
ultimos_indicadores = ultimos_indicadores.reset_index(drop=True)
predictions_df = predictions_df.reset_index(drop=True)
merged_last_10 = pd.concat([predictions_df, ultimos_indicadores], axis=1)
display(merged_last_10)

,Open,High,Low,Close,Open_time,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR
0,6.84,6.99,6.56,6.77,2024-04-30,6.99,7.08,7.50,6.99,6.47,41.62,-0.18,-0.25,26.94,62.40,65.75,-44.90,0.47
1,6.84,6.91,6.70,6.70,2024-05-01,6.96,7.04,7.48,6.96,6.44,40.47,-0.19,-0.24,26.92,55.73,61.77,-47.27,0.45
2,6.89,6.83,6.82,6.63,2024-05-02,6.92,7.00,7.41,6.92,6.44,39.33,-0.21,-0.23,26.91,45.85,54.66,-49.34,0.43
3,6.91,6.76,6.80,6.56,2024-05-03,6.89,6.96,7.37,6.89,6.40,38.20,-0.22,-0.23,26.95,40.87,47.49,-68.29,0.41
4,6.93,6.69,6.95,6.49,2024-05-04,6.86,6.92,7.38,6.86,6.35,37.09,-0.23,-0.23,26.98,34.52,40.41,-84.30,0.41
5,7.00,6.62,6.95,6.42,2024-05-05,6.84,6.87,7.39,6.84,6.29,35.99,-0.24,-0.23,27.01,15.87,30.42,-99.98,0.41
6,6.95,6.55,7.03,6.36,2024-05-06,6.82,6.82,7.41,6.82,6.23,34.91,-0.26,-0.24,27.03,-3.49,15.63,-89.48,0.42
7,6.94,6.49,7.06,6.30,2024-05-07,6.80,6.77,7.43,6.80,6.16,33.85,-0.27,-0.24,27.06,-22.03,-3.22,-87.64,0.44
8,6.92,6.42,7.31,6.24,2024-05-08,6.77,6.72,7.45,6.77,6.09,32.80,-0.28,-0.25,27.08,-28.79,-18.10,-66.64,0.49
9,6.91,6.36,7.40,6.18,2024-05-09,6.75,6.67,7.47,6.75,6.03,31.78,-0.29,-0.26,27.11,-35.36,-28.73,-66.68,0.53


In [277]:
merged_last_10.to_csv('autoTS_models/result.csv', index=False)